In [57]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
import pandas as pd
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn import metrics
import optuna
from sklearn.svm import SVC
from sklearn.utils import resample
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import KeyedVectors
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score


In [23]:
#carregar dataset
df = pd.read_csv('factnews_dataset.csv')
df.isnull().sum()

file          0
id_sente      0
id_article    0
domain        0
year          0
sentences     0
classe        0
dtype: int64

# Pré-processamento

divisão dos dados

In [24]:
'''
#criar grupo treino, validação e teste
train_df, test_df = train_test_split(df, test_size=0.3, random_state=42, stratify=df['classe'])
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42, stratify=train_df['classe'])

#guardar conjuntos em memória
train_df.to_csv("train.csv", index=False) 
val_df.to_csv("val.csv", index=False)
test_df.to_csv("test.csv", index=False)
'''

'\n#criar grupo treino, validação e teste\ntrain_df, test_df = train_test_split(df, test_size=0.3, random_state=42, stratify=df[\'classe\'])\ntrain_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42, stratify=train_df[\'classe\'])\n\n#guardar conjuntos em memória\ntrain_df.to_csv("train.csv", index=False) \nval_df.to_csv("val.csv", index=False)\ntest_df.to_csv("test.csv", index=False)\n'

balanceamento dos dados

In [58]:
train= pd.read_csv('train.csv')
val= pd.read_csv('val.csv')
test= pd.read_csv('test.csv')

train_oversampling= pd.read_csv('train_oversampling.csv')
val_oversampling= pd.read_csv('val_oversampling.csv')
test_oversampling= pd.read_csv('test_oversampling.csv')

y_train= train['classe']
y_train_oversampling= train_oversampling['classe']

y_val=val['classe']
y_val_oversampling = val_oversampling['classe']

y_test= test['classe']
y_test_oversampling = test_oversampling['classe']

In [59]:
# Contagem das classes em cada conjunto
print ("conjunto de treino: ",y_train.value_counts().sort_index())
print("\nconjunto de validação: ",y_val.value_counts().sort_index())
print("\nconjunto de teste: ", y_test.value_counts().sort_index())
print("\nconjunto de treino + exemplos: ", y_train_oversampling.value_counts().sort_index())
print("\nconjunto de val + exemplos: ", y_val_oversampling.value_counts().sort_index())
print("\nconjunto de teste + exemplos: ", y_test_oversampling.value_counts().sort_index())



conjunto de treino:  classe
-1     779
 0    2375
 1     312
Name: count, dtype: int64

conjunto de validação:  classe
-1    195
 0    594
 1     78
Name: count, dtype: int64

conjunto de teste:  classe
-1     417
 0    1273
 1     168
Name: count, dtype: int64

conjunto de treino + exemplos:  classe
-1    2375
 0    2375
 1    2375
Name: count, dtype: int64

conjunto de val + exemplos:  classe
-1    594
 0    594
 1    594
Name: count, dtype: int64

conjunto de teste + exemplos:  classe
-1    1273
 0    1273
 1    1273
Name: count, dtype: int64


In [60]:
from sklearn.utils import resample

def balancear_classes(conjunto):
  # Separar cada classe
  class_min = conjunto[conjunto['classe'] == -1]
  class_med = conjunto[conjunto['classe'] == 1]
  class_max = conjunto[conjunto['classe'] == 0]

  # Número da classe maior
  n_samples = len(class_max)

  # Replicar as classes menores até o tamanho da maior
  class_min_upsampled = resample(class_min, 
                              replace=True, 
                              n_samples=n_samples, 
                              random_state=42)

  class_med_upsampled = resample(class_med, 
                              replace=True, 
                              n_samples=n_samples, 
                            random_state=42)

  conjunto_oversampled = pd.concat([class_min_upsampled, class_med_upsampled, class_max])

  # baralhar o conjunto 
  conjunto_oversampled = conjunto_oversampled.sample(frac=1, random_state=42).reset_index(drop=True)

  print(conjunto_oversampled['classe'].value_counts())

  return conjunto_oversampled

#train_oversampling.to_csv('train_oversampling.csv', index=False)
#val_oversampling = balancear_classes(val)
#test_oversampling= balancear_classes(test)

#val_oversampling.to_csv('val_oversampling.csv', index=False)
#test_oversampling.to_csv('test_oversampling.csv', index=False)


tokeinização

In [61]:
stop_words = set(stopwords.words('portuguese'))
def tokeinizar(train, val, test):

    #criar coluna tokens
    train['tokens'] = train['sentences'].apply(lambda x: nltk.word_tokenize(str(x)))
    val['tokens']   = val['sentences'].apply(lambda x: nltk.word_tokenize(str(x)))
    test['tokens'] = test['sentences'].apply(lambda x: nltk.word_tokenize(str(x)))
    train_oversampling['tokens'] = train_oversampling['sentences'].apply(lambda x: nltk.word_tokenize(str(x)))

    #remover stopwords
    train['tokens'] = train['tokens'].apply(lambda toks: [t for t in toks if t.lower() not in stop_words])
    val['tokens']   = val['tokens'].apply(lambda toks: [t for t in toks if t.lower() not in stop_words])
    test['tokens']  = test['tokens'].apply(lambda toks: [t for t in toks if t.lower() not in stop_words])
    train_oversampling['tokens'] = train_oversampling['tokens'].apply(lambda toks: [t for t in toks if t.lower() not in stop_words])

    #matizar
    #nltk . download ('rslp')
    stemmer = nltk . stem . RSLPStemmer ()
    train['stems'] = train['tokens'].apply(lambda tokens: [stemmer.stem(token) for token in tokens])
    val['stems']   = val['tokens'].apply(lambda tokens: [stemmer.stem(token) for token in tokens])
    test['stems'] = test['tokens'].apply(lambda tokens: [stemmer.stem(token) for token in tokens])
    train_oversampling['stems'] = train_oversampling['tokens'].apply(lambda tokens: [stemmer.stem(token) for token in tokens])

    nltk . download ('rslp')
    stemmer = nltk . stem . RSLPStemmer ()
    print(train['stems'].head())

    return train, val, test

train, val, test = tokeinizar(train, val, test)
train_oversampling, val_oversampling, test_oversampling = tokeinizar(train_oversampling, val_oversampling, test_oversampling)

[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\didia\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!


0    [neym, ,, part, 20, novembr, vai, lider, seleç...
1    [forç, internac, transform, quest, taiwan, ass...
2    [alianç, mai, quant, parlament, eleit, nome, p...
3    [ger, mort, '', ,, declar, cabr, ,, 16ª, entre...
4    [bisp, barr, (, ba, ), ,, dom, luiz, flávi, ca...
Name: stems, dtype: object
0    [qued, ocorr, aliment, pux, cord, sent, contr,...
1    [repercuss, nega, cas, fez, bolsonar, ,, candi...
2    [brasil, lav, alm, após, decepcion, empat, col...
3    [final, encontr, ,, sen, tucan, diss, esp, gov...
4    [fiz, compromiss, líd, bas, hoj, líd, opos, ,,...
Name: stems, dtype: object


[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\didia\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!


# Construção dos modelos de representação

## TF-IDF - fred

In [51]:
def TF_IDF(train, val, test):
    tf_vect = TfidfVectorizer(ngram_range =(1, 2), min_df =3, max_df =0.5, max_features =500)

        #juntar os tokens matizados em frases novvamente 
    text_train = train['stems'].apply(lambda tokens: ' '.join(tokens))
    text_val   = val['stems'].apply(lambda tokens: ' '.join(tokens))
    text_test = test['stems'].apply(lambda tokens: ' '.join(tokens))

    X_train = tf_vect.fit_transform(text_train)
    X_val   = tf_vect.transform(text_val)
    X_test  = tf_vect.transform(text_test)
    return X_train, X_val, X_test

In [52]:
X_train_tfidf_vec, X_val_tfidf_vec, X_test_tfidf_vec = TF_IDF(train,val, test)
X_train_oversampling_tfidf_vec, X_val_oversampling_tfidf_vec, X_test_oversampling_tfidf_vec = TF_IDF(train_oversampling, val_oversampling, test_oversampling)

## CountVectorizer -didi

In [53]:
import optuna
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

def objective_count_vec(trial, train, val):
    # Parâmetros a otimizar
    max_features = trial.suggest_int("max_features", 500, 5000)
    ngram_min = trial.suggest_int("ngram_min", 1, 2)
    ngram_max = trial.suggest_int("ngram_max", ngram_min, 3)  # garante ngram_max >= ngram_min

    # CountVectorizer com parâmetros sugeridos
    c_vect = CountVectorizer(
        token_pattern=r"(?u)\b\w+\b|[^\w\s]",
        ngram_range=(ngram_min, ngram_max),
        strip_accents='unicode',
        max_features=max_features
    )

    # Transformar textos
    text_train = train['stems'].apply(lambda tokens: ' '.join(tokens))
    text_val   = val['stems'].apply(lambda tokens: ' '.join(tokens))

    X_train = c_vect.fit_transform(text_train)
    X_val   = c_vect.transform(text_val)

    y_train = train['classe'] 
    y_val   = val['classe']

    # Treinar Logistic Regression
    model = LogisticRegression(max_iter=500)
    model.fit(X_train, y_train)
    preds = model.predict(X_val)

    # Métrica: F1 macro
    score = f1_score(y_val, preds, average='macro')
    return score 

def count_vec(train, val, test):

    study = optuna.create_study(direction="maximize")
    study.optimize(lambda trial: objective_count_vec(trial, train,val),
                   n_trials=30, n_jobs=-1, show_progress_bar=False)
    
    best_params=study.best_params
    print("Melhores parâmetros:", study.best_params)
    print("Melhor F1:", study.best_value)
    
    c_vect = CountVectorizer(
        token_pattern=r"(?u)\b\w+\b|[^\w\s]",
        ngram_range=(best_params['ngram_min'], best_params['ngram_max']),
        strip_accents='unicode',
        max_features=best_params['max_features']
    )

    # Transformar textos
    text_train = train['stems'].apply(lambda tokens: ' '.join(tokens))
    text_val = val['stems'].apply(lambda tokens: ' '.join(tokens))
    text_test = test['stems'].apply(lambda tokens: ' '.join(tokens))

    # Fit no treino original, transform nos outros conjuntos
    X_train = c_vect.fit_transform(text_train)
    X_val = c_vect.transform(text_val)
    X_test = c_vect.transform(text_test)

    return X_train, X_val, X_test

In [54]:
X_train_count_vec, X_val_count_vec, X_test_count_vec = count_vec(train, val, test)
X_train_oversampling_count_vec, X_val_oversampling_count_vec, X_test_oversampling_count_vec= count_vec(train_oversampling, val_oversampling, test_oversampling)

[I 2025-11-12 11:54:10,779] A new study created in memory with name: no-name-17bbd21c-33a7-4170-baa2-6d698f6071d8
[I 2025-11-12 11:54:12,898] Trial 4 finished with value: 0.5394926283725304 and parameters: {'max_features': 1111, 'ngram_min': 2, 'ngram_max': 2}. Best is trial 4 with value: 0.5394926283725304.
[I 2025-11-12 11:54:13,260] Trial 3 finished with value: 0.5241011943139603 and parameters: {'max_features': 942, 'ngram_min': 2, 'ngram_max': 3}. Best is trial 4 with value: 0.5394926283725304.
[I 2025-11-12 11:54:13,428] Trial 0 finished with value: 0.5404637981440216 and parameters: {'max_features': 958, 'ngram_min': 2, 'ngram_max': 2}. Best is trial 0 with value: 0.5404637981440216.
[I 2025-11-12 11:54:13,489] Trial 6 finished with value: 0.5415784360496737 and parameters: {'max_features': 1044, 'ngram_min': 2, 'ngram_max': 2}. Best is trial 6 with value: 0.5415784360496737.
[I 2025-11-12 11:54:14,031] Trial 7 finished with value: 0.5455935687298529 and parameters: {'max_featur

Melhores parâmetros: {'max_features': 3764, 'ngram_min': 1, 'ngram_max': 3}
Melhor F1: 0.6736598517979013


[I 2025-11-12 11:54:25,816] A new study created in memory with name: no-name-72e3739d-9d69-442a-98bd-1bf07bedfbb0
[I 2025-11-12 11:54:29,727] Trial 2 finished with value: 0.5577539649309203 and parameters: {'max_features': 1679, 'ngram_min': 2, 'ngram_max': 2}. Best is trial 2 with value: 0.5577539649309203.
[I 2025-11-12 11:54:30,074] Trial 7 finished with value: 0.5541461599067071 and parameters: {'max_features': 1299, 'ngram_min': 2, 'ngram_max': 2}. Best is trial 2 with value: 0.5577539649309203.
[I 2025-11-12 11:54:30,594] Trial 0 finished with value: 0.5433609290553749 and parameters: {'max_features': 3199, 'ngram_min': 2, 'ngram_max': 2}. Best is trial 2 with value: 0.5577539649309203.
[I 2025-11-12 11:54:31,027] Trial 3 finished with value: 0.5343507359623395 and parameters: {'max_features': 4072, 'ngram_min': 2, 'ngram_max': 3}. Best is trial 2 with value: 0.5577539649309203.
[I 2025-11-12 11:54:31,747] Trial 1 finished with value: 0.6475990302424601 and parameters: {'max_feat

Melhores parâmetros: {'max_features': 4173, 'ngram_min': 1, 'ngram_max': 1}
Melhor F1: 0.6882682503967931


## Glove -fred

In [45]:


glove = KeyedVectors.load_word2vec_format('glove_s100.txt')

def vetor_frase_glove(tokens):
    vecs = [glove[w] for w in tokens if w in glove]
    return np.mean(vecs, axis=0) if vecs else np.zeros(glove.vector_size)

def prep_dados(dados, coluna):
    return np.vstack([vetor_frase_glove(tokens) for tokens in dados[coluna]])

def objective(trial):
    c = trial.suggest_float("C", 1e-4, 1e2, log=True)
    model = SVC(C=c, kernel="linear")
    X_train = prep_dados(train, "tokens")
    X_val = prep_dados(val, "tokens")
    y_train = train["classe"].values 
    y_val = val["classe"].values      
    model.fit(X_train, y_train)
    preds = model.predict(X_val)
    return f1_score(y_val, preds, average="macro")


study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=30)

print("Melhores parâmetros:", study.best_params)
print("Melhor F1:", study.best_value)

X_train_glove = prep_dados(train, "tokens")
X_val_glove = prep_dados(val, "tokens")
X_test_glove = prep_dados(test, "tokens")


[I 2025-11-12 11:44:39,676] A new study created in memory with name: no-name-bb669a83-cc39-4c81-bae1-811f97aee124
[I 2025-11-12 11:44:40,977] Trial 0 finished with value: 0.5212138168290926 and parameters: {'C': 11.99024030254317}. Best is trial 0 with value: 0.5212138168290926.
[I 2025-11-12 11:44:42,524] Trial 1 finished with value: 0.5187624391503963 and parameters: {'C': 14.75664584821376}. Best is trial 0 with value: 0.5212138168290926.
[I 2025-11-12 11:44:44,090] Trial 2 finished with value: 0.5187624391503963 and parameters: {'C': 14.254985500190031}. Best is trial 0 with value: 0.5212138168290926.
[I 2025-11-12 11:45:10,690] Trial 3 finished with value: 0.5202543013802231 and parameters: {'C': 37.31758082756325}. Best is trial 0 with value: 0.5212138168290926.
[I 2025-11-12 11:45:12,216] Trial 4 finished with value: 0.5187624391503963 and parameters: {'C': 13.722247169063323}. Best is trial 0 with value: 0.5212138168290926.
[I 2025-11-12 11:45:14,731] Trial 5 finished with valu

Melhores parâmetros: {'C': 0.3092902427335214}
Melhor F1: 0.5279850519904391


## WORD2VEC -anaana

fontes : 
-   https://sites.google.com/view/nilc-usp/resources-and-tools?authuser=0
-   https://huggingface.co/nilc-nlp/word2vec-skip-gram-300d

In [55]:
#%pip install huggingface_hub
#%pip install hf_hub_download
#%pip install safetensors
from huggingface_hub import hf_hub_download
from safetensors.numpy import load_file

def carregar_word2vec():
    path = hf_hub_download(repo_id="nilc-nlp/word2vec-skip-gram-300d",
                           filename="embeddings.safetensors")

    data = load_file(path)
    vectors = data["embeddings"]

    vocab_path = hf_hub_download(repo_id="nilc-nlp/word2vec-skip-gram-300d",
                                 filename="vocab.txt")
    with open(vocab_path) as f:
        vocab = [w.strip() for w in f]

    print(vectors.shape)

    model = KeyedVectors(vector_size=vectors.shape[1])
    model.add_vectors(vocab, vectors)
    model.fill_norms()

    return model

def vetor_frase(model, frase):
    vectors = [model[w] for w in frase if w in model]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(model.vector_size)

def word2vec(train, val, test, coluna, model):
    X_train = np.vstack([vetor_frase(model, tokens) for tokens in train[coluna]])
    X_val   = np.vstack([vetor_frase(model, tokens) for tokens in val[coluna]])
    X_test  = np.vstack([vetor_frase(model, tokens) for tokens in test[coluna]])

    return X_train, X_val, X_test

In [56]:
model = carregar_word2vec()
X_train_word2vec, X_val_word2vec, X_test_word2vec = word2vec(train, val, test, 'stems', model)

(929606, 300)


## WORD2VEC treinado com os nossos dados

In [ ]:
import numpy as np
import optuna
from gensim.models import Word2Vec
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import f1_score

#vetor médio da frase
def vetor_frase_w2v(tokens, model):
    vecs = [model.wv[w] for w in tokens if w in model.wv]
    return np.mean(vecs, axis=0) if vecs else np.zeros(model.vector_size)

# matriz dos embeddings
def prep_dados_w2v(dados, coluna, model):
    return np.vstack([vetor_frase_w2v(tokens, model) for tokens in dados[coluna]])

def objective_w2v(trial, train, val):
    # Hiperparâmetros 
    vector_size = trial.suggest_int("vector_size", 50, 300)
    window = trial.suggest_int("window", 3, 10)
    min_count = trial.suggest_int("min_count", 1, 5)
    sg = trial.suggest_categorical("sg", [0, 1])  # 0 = CBOW, 1 = Skip-gram
    epochs = trial.suggest_int("epochs", 5, 20)
    C = trial.suggest_float("C", 1e-3, 10, log=True)
    use_scaler = trial.suggest_categorical("use_scaler", [True, False])

    # Treina Word2Vec com os parâmetros sugeridos
    sentences = train["tokens"]
    model = Word2Vec(
        sentences=sentences,
        vector_size=vector_size,
        window=window,
        min_count=min_count,
        sg=sg,
        workers=4,
        epochs=epochs
    )

    # Transformar dados nos embeddings médios
    X_train = prep_dados_w2v(train, "tokens", model)
    X_val = prep_dados_w2v(val, "tokens", model)
    y_train = train["classe"].values
    y_val = val["classe"].values

    if use_scaler:
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_val = scaler.transform(X_val)

    # Modelo de classificação (Logistic Regression)
    clf = LogisticRegression(C=C, max_iter=500)
    clf.fit(X_train, y_train)
    preds = clf.predict(X_val)

    return f1_score(y_val, preds, average="macro")

def word2vec_vec(train, val, test):
    study = optuna.create_study(direction="maximize")
    study.optimize(lambda trial: objective_w2v(trial, train, val),
                   n_trials=30, n_jobs=-1, show_progress_bar=False)

    best_params = study.best_params
    print("Melhores parâmetros:", best_params)
    print("Melhor F1:", study.best_value)

    best_model = Word2Vec(
        sentences=train["tokens"],
        vector_size=best_params["vector_size"],
        window=best_params["window"],
        min_count=best_params["min_count"],
        sg=best_params["sg"],
        workers=4,
        epochs=best_params["epochs"]
    )

    # Criar embeddings médios para todos os conjuntos
    X_train = prep_dados_w2v(train, "tokens", best_model)
    X_val = prep_dados_w2v(val, "tokens", best_model)
    X_test = prep_dados_w2v(test, "tokens", best_model)

    if best_params["use_scaler"]:
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_val = scaler.transform(X_val)
        X_test = scaler.transform(X_test)

    return X_train, X_val, X_test


In [63]:
X_train_w2v_trained, X_val_w2v_trained, X_test_w2v_trained = word2vec_vec(train, val, test)
X_train_over_w2v_trained, X_val_over_w2v_trained, X_test_over_w2v_trained= word2vec_vec(train_oversampling, val_oversampling, test_oversampling)

[I 2025-11-12 12:04:54,426] A new study created in memory with name: no-name-36755386-72e5-4e94-be24-85466841e32e
[I 2025-11-12 12:05:03,126] Trial 4 finished with value: 0.3565951748417892 and parameters: {'vector_size': 119, 'window': 5, 'min_count': 2, 'sg': 0, 'epochs': 8, 'C': 0.0313451146132693, 'use_scaler': False}. Best is trial 4 with value: 0.3565951748417892.
[I 2025-11-12 12:05:07,093] Trial 6 finished with value: 0.43940101733624 and parameters: {'vector_size': 295, 'window': 3, 'min_count': 1, 'sg': 1, 'epochs': 6, 'C': 1.8554523894987398, 'use_scaler': False}. Best is trial 6 with value: 0.43940101733624.
[I 2025-11-12 12:05:07,136] Trial 0 finished with value: 0.34219539771215746 and parameters: {'vector_size': 273, 'window': 8, 'min_count': 1, 'sg': 1, 'epochs': 12, 'C': 0.010678912790070036, 'use_scaler': False}. Best is trial 6 with value: 0.43940101733624.
[I 2025-11-12 12:05:07,384] Trial 7 finished with value: 0.3008971704623879 and parameters: {'vector_size': 155

Melhores parâmetros: {'vector_size': 180, 'window': 9, 'min_count': 2, 'sg': 1, 'epochs': 15, 'C': 0.1572830696728889, 'use_scaler': True}
Melhor F1: 0.5422920316466545


[I 2025-11-12 12:06:14,739] A new study created in memory with name: no-name-b54657f4-60d0-4414-9169-6184f4c7aa32
[I 2025-11-12 12:06:38,191] Trial 0 finished with value: 0.5508341994788306 and parameters: {'vector_size': 261, 'window': 6, 'min_count': 1, 'sg': 0, 'epochs': 12, 'C': 0.05089614895761777, 'use_scaler': False}. Best is trial 0 with value: 0.5508341994788306.
[I 2025-11-12 12:06:38,374] Trial 6 finished with value: 0.6081188033572597 and parameters: {'vector_size': 99, 'window': 6, 'min_count': 3, 'sg': 0, 'epochs': 20, 'C': 0.006102284801883379, 'use_scaler': True}. Best is trial 6 with value: 0.6081188033572597.
[I 2025-11-12 12:06:38,819] Trial 2 finished with value: 0.6187006203543386 and parameters: {'vector_size': 217, 'window': 7, 'min_count': 2, 'sg': 1, 'epochs': 18, 'C': 0.0034133770832388373, 'use_scaler': True}. Best is trial 2 with value: 0.6187006203543386.
[I 2025-11-12 12:06:42,193] Trial 5 finished with value: 0.6344097245429964 and parameters: {'vector_si

Melhores parâmetros: {'vector_size': 125, 'window': 4, 'min_count': 4, 'sg': 1, 'epochs': 6, 'C': 7.623812439288696, 'use_scaler': False}
Melhor F1: 0.6658890527325335


In [ ]:
'''
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
certos = (test_df['classe'] == test_df['Previsto4']).sum()
total = len(test_df)

matriz = confusion_matrix(test_df['classe'], test_df['Previsto4'], labels=classes)

disp = ConfusionMatrixDisplay(confusion_matrix=matriz, display_labels=classes)
disp.plot()
plt.title("Matriz de confusão do teste")
plt.show()

from sklearn.metrics import classification_report

report = classification_report(test_df['classe'], test_df['Previsto4'], labels=classes, digits=4)
print(report)
'''

# Aplicar modelos de classificação

## Naive Bayes 

-   Multinomial

In [68]:
def objective_naive_MN(trial, X_train, X_val, y_train, y_val):
    # Hiperparâmetros a otimizar
    alpha = trial.suggest_float("alpha", 1e-3, 2.0, log=True)
    
    # Modelo
    clf = MultinomialNB(alpha=alpha)
    clf.fit(X_train, y_train)
    
    y_val_pred = clf.predict(X_val)
    f1 = f1_score(y_val, y_val_pred, average='weighted')  
    
    return f1

def Naive_Bayes_MN(X_train, X_val, X_test, y_train, y_val, n_trials=30):
    study = optuna.create_study(direction="maximize")
    study.optimize(lambda trial: objective_naive_MN(trial, X_train, X_val, y_train, y_val),
                   n_trials=n_trials, n_jobs=-1, show_progress_bar=False)
    
    print("Melhores hiperparâmetros encontrados:")
    print(study.best_params)
    print(f"Melhor F1 (validação): {study.best_value:.4f}")
    
    # Treinar o modelo final com o melhor alpha
    best_model = MultinomialNB(**study.best_params)
    best_model.fit(X_train, y_train)
    
    # Previsão no conjunto de teste
    y_test_pred = best_model.predict(X_test)
    
    return y_test_pred

In [72]:
from sklearn.metrics import f1_score

def objective_naive_GS(trial, X_train, X_val, y_train, y_val):
    # Definir o hiperparâmetro a otimizar
    var_smoothing = trial.suggest_float("var_smoothing", 1e-12, 1e-3, log=True)
    # Criar o modelo
    model = GaussianNB(var_smoothing=var_smoothing)
    # Treinar
    model.fit(X_train, y_train)
    # Avaliar no conjunto de validação
    y_pred = model.predict(X_val)
    f1 = f1_score(y_val, y_pred, average="macro")

    return f1

def naive_bayes_gs(X_train, X_val, X_test, y_train, y_val, n_trials=30):
    study = optuna.create_study(direction="maximize")
    study.optimize(lambda trial: objective_naive_GS(trial, X_train, X_val, y_train, y_val),
                   n_trials=n_trials, n_jobs=-1, show_progress_bar=False)
    print("Melhores hiperparâmetros encontrados:")
    print(study.best_params)
    # Treinar o melhor modelo com os melhores parâmetros
    best_model = GaussianNB(**study.best_params)
    best_model.fit(X_train, y_train)
    # Avaliar no conjunto de teste
    y_test_pred = best_model.predict(X_test)

    return y_test_pred

In [79]:
# Count Vectorizer
print ("----COUNT VECTORIZER ----")
count_vec_predicted_NB = Naive_Bayes_MN(X_train_count_vec,X_val_count_vec,X_test_count_vec,y_train,y_val)
print(metrics.classification_report(y_test, count_vec_predicted_NB))
print(metrics.confusion_matrix(y_test, count_vec_predicted_NB))

print("\ncom oversampling")
count_vec_over_predicted_NB = Naive_Bayes_MN(X_train_oversampling_count_vec,X_val_oversampling_count_vec,X_test_oversampling_count_vec,y_train_oversampling,y_val_oversampling)
print(metrics.classification_report(y_test_oversampling, count_vec_over_predicted_NB))
print(metrics.confusion_matrix(y_test_oversampling, count_vec_over_predicted_NB))

[I 2025-11-12 12:31:52,934] A new study created in memory with name: no-name-6bb26185-63ce-47d5-9ce9-1191a172c223
[I 2025-11-12 12:31:52,976] Trial 0 finished with value: 0.7976716638673003 and parameters: {'alpha': 0.1223322375633379}. Best is trial 0 with value: 0.7976716638673003.
[I 2025-11-12 12:31:52,991] Trial 3 finished with value: 0.7946176868337981 and parameters: {'alpha': 0.03262650686039077}. Best is trial 0 with value: 0.7976716638673003.
[I 2025-11-12 12:31:53,005] Trial 2 finished with value: 0.7985662170424201 and parameters: {'alpha': 0.10800825970666929}. Best is trial 2 with value: 0.7985662170424201.
[I 2025-11-12 12:31:53,019] Trial 5 finished with value: 0.7930916489767073 and parameters: {'alpha': 0.004119816044564838}. Best is trial 2 with value: 0.7985662170424201.
[I 2025-11-12 12:31:53,024] Trial 6 finished with value: 0.8020100569340904 and parameters: {'alpha': 0.01520093538677455}. Best is trial 6 with value: 0.8020100569340904.
[I 2025-11-12 12:31:53,024

----COUNT VECTORIZER ----


[I 2025-11-12 12:31:53,122] Trial 14 finished with value: 0.7945180721494238 and parameters: {'alpha': 0.001715682252774056}. Best is trial 6 with value: 0.8020100569340904.
[I 2025-11-12 12:31:53,135] Trial 15 finished with value: 0.7964809039890579 and parameters: {'alpha': 0.026248104614382546}. Best is trial 6 with value: 0.8020100569340904.
[I 2025-11-12 12:31:53,156] Trial 13 finished with value: 0.805579226930671 and parameters: {'alpha': 0.6771193627784311}. Best is trial 13 with value: 0.805579226930671.
[I 2025-11-12 12:31:53,185] Trial 16 finished with value: 0.7996507809270847 and parameters: {'alpha': 0.05391977651804105}. Best is trial 13 with value: 0.805579226930671.
[I 2025-11-12 12:31:53,189] Trial 18 finished with value: 0.8009374009448466 and parameters: {'alpha': 1.4845229485016158}. Best is trial 13 with value: 0.805579226930671.
[I 2025-11-12 12:31:53,196] Trial 17 finished with value: 0.799141686947886 and parameters: {'alpha': 0.021414739422497126}. Best is tri

Melhores hiperparâmetros encontrados:
{'alpha': 0.7379387636107038}
Melhor F1 (validação): 0.8065
              precision    recall  f1-score   support

          -1       0.71      0.79      0.75       417
           0       0.86      0.84      0.85      1273
           1       0.32      0.30      0.31       168

    accuracy                           0.78      1858
   macro avg       0.63      0.64      0.64      1858
weighted avg       0.78      0.78      0.78      1858

[[ 328   77   12]
 [ 112 1065   96]
 [  19   98   51]]

com oversampling


[I 2025-11-12 12:31:53,478] Trial 10 finished with value: 0.6490286609024564 and parameters: {'alpha': 0.26220833641962477}. Best is trial 7 with value: 0.6897191407776981.
[I 2025-11-12 12:31:53,483] Trial 9 finished with value: 0.5679050846561783 and parameters: {'alpha': 0.005541186937094919}. Best is trial 7 with value: 0.6897191407776981.
[I 2025-11-12 12:31:53,502] Trial 12 finished with value: 0.5658950559180109 and parameters: {'alpha': 0.0036777857155456863}. Best is trial 7 with value: 0.6897191407776981.
[I 2025-11-12 12:31:53,506] Trial 11 finished with value: 0.6337039368245941 and parameters: {'alpha': 0.11136808503065809}. Best is trial 7 with value: 0.6897191407776981.
[I 2025-11-12 12:31:53,513] Trial 13 finished with value: 0.6184702959153636 and parameters: {'alpha': 0.043175375847273656}. Best is trial 7 with value: 0.6897191407776981.
[I 2025-11-12 12:31:53,523] Trial 14 finished with value: 0.5805165232356159 and parameters: {'alpha': 0.008404783134965973}. Best i

Melhores hiperparâmetros encontrados:
{'alpha': 1.9321130497623302}
Melhor F1 (validação): 0.6969
              precision    recall  f1-score   support

          -1       0.76      0.80      0.78      1273
           0       0.55      0.76      0.64      1273
           1       0.70      0.40      0.51      1273

    accuracy                           0.65      3819
   macro avg       0.67      0.65      0.64      3819
weighted avg       0.67      0.65      0.64      3819

[[1013  197   63]
 [ 142  970  161]
 [ 170  588  515]]


In [ ]:
print ("----TF-IDF ----")
tfidf_vec_predicted_NB= Naive_Bayes_MN(X_train_tfidf_vec, X_val_tfidf_vec, X_test_tfidf_vec, y_train, y_val) 
print(metrics.classification_report(y_test, tfidf_vec_predicted_NB))
print(metrics.confusion_matrix(y_test, tfidf_vec_predicted_NB))

print("\n com oversampling")
tfidf_vec_oversampling_predicted_NB= Naive_Bayes_MN(X_train_oversampling_tfidf_vec, X_val_oversampling_tfidf_vec, X_test_oversampling_tfidf_vec, y_train_oversampling, y_val_oversampling) 
print(metrics.classification_report(y_test_oversampling, tfidf_vec_oversampling_predicted_NB))
print(metrics.confusion_matrix(y_test_oversampling, tfidf_vec_oversampling_predicted_NB))

[I 2025-11-12 12:38:16,081] A new study created in memory with name: no-name-bfdfbd2e-9e73-4b38-9cb1-49c0856e5ec2
[I 2025-11-12 12:38:16,130] Trial 2 finished with value: 0.6641849232600153 and parameters: {'alpha': 0.12720207236525366}. Best is trial 2 with value: 0.6641849232600153.
[I 2025-11-12 12:38:16,135] Trial 1 finished with value: 0.6708793861750012 and parameters: {'alpha': 0.016997523960351438}. Best is trial 1 with value: 0.6708793861750012.
[I 2025-11-12 12:38:16,135] Trial 0 finished with value: 0.6708793861750012 and parameters: {'alpha': 0.00232989968046434}. Best is trial 1 with value: 0.6708793861750012.
[I 2025-11-12 12:38:16,149] Trial 3 finished with value: 0.6531770790607648 and parameters: {'alpha': 0.5831367672638899}. Best is trial 1 with value: 0.6708793861750012.
[I 2025-11-12 12:38:16,157] Trial 5 finished with value: 0.6708793861750012 and parameters: {'alpha': 0.01762067130083276}. Best is trial 1 with value: 0.6708793861750012.
[I 2025-11-12 12:38:16,157

----TF-IDF ----


[I 2025-11-12 12:38:16,274] Trial 22 finished with value: 0.6708793861750012 and parameters: {'alpha': 0.0012268182182498754}. Best is trial 1 with value: 0.6708793861750012.
[I 2025-11-12 12:38:16,308] Trial 19 finished with value: 0.6708793861750012 and parameters: {'alpha': 0.0011595387736744551}. Best is trial 1 with value: 0.6708793861750012.
[I 2025-11-12 12:38:16,297] Trial 23 finished with value: 0.6708793861750012 and parameters: {'alpha': 0.0014033284410298263}. Best is trial 1 with value: 0.6708793861750012.
[I 2025-11-12 12:38:16,299] Trial 20 finished with value: 0.6708793861750012 and parameters: {'alpha': 0.0016712429885975158}. Best is trial 1 with value: 0.6708793861750012.
[I 2025-11-12 12:38:16,289] Trial 25 finished with value: 0.6708793861750012 and parameters: {'alpha': 0.037496674139359656}. Best is trial 1 with value: 0.6708793861750012.
[I 2025-11-12 12:38:16,322] Trial 18 finished with value: 0.6708793861750012 and parameters: {'alpha': 0.0011187614242195805}.

Melhores hiperparâmetros encontrados:
{'alpha': 0.016997523960351438}
Melhor F1 (validação): 0.6709
              precision    recall  f1-score   support

          -1       0.69      0.24      0.36       417
           0       0.72      0.97      0.83      1273
           1       0.67      0.01      0.02       168

    accuracy                           0.72      1858
   macro avg       0.69      0.41      0.40      1858
weighted avg       0.71      0.72      0.65      1858

[[ 101  316    0]
 [  34 1238    1]
 [  12  154    2]]

 com oversampling


[I 2025-11-12 12:38:16,549] Trial 16 finished with value: 0.5889413734124402 and parameters: {'alpha': 0.0028210653500723966}. Best is trial 1 with value: 0.5905541674964261.
[I 2025-11-12 12:38:16,569] Trial 18 finished with value: 0.5900101597555611 and parameters: {'alpha': 0.2402142313967455}. Best is trial 1 with value: 0.5905541674964261.
[I 2025-11-12 12:38:16,569] Trial 17 finished with value: 0.5895110691778072 and parameters: {'alpha': 0.4501046650982607}. Best is trial 1 with value: 0.5905541674964261.
[I 2025-11-12 12:38:16,599] Trial 20 finished with value: 0.5900407331930773 and parameters: {'alpha': 0.3736036273201538}. Best is trial 1 with value: 0.5905541674964261.
[I 2025-11-12 12:38:16,616] Trial 19 finished with value: 0.5894658743874276 and parameters: {'alpha': 0.2761792071375014}. Best is trial 1 with value: 0.5905541674964261.
[I 2025-11-12 12:38:16,635] Trial 22 finished with value: 0.5900101597555611 and parameters: {'alpha': 0.2617407943725502}. Best is trial

Melhores hiperparâmetros encontrados:
{'alpha': 0.17223707269542368}
Melhor F1 (validação): 0.5906
              precision    recall  f1-score   support

          -1       0.63      0.61      0.62      1273
           0       0.51      0.64      0.57      1273
           1       0.56      0.45      0.50      1273

    accuracy                           0.57      3819
   macro avg       0.57      0.57      0.56      3819
weighted avg       0.57      0.57      0.56      3819

[[773 307 193]
 [207 812 254]
 [239 459 575]]


In [ ]:
#Word2vec
print ("----WORD2VEC ----")
word2vec_predicted_NB= naive_bayes_gs(X_train_word2vec, X_val_word2vec, X_test_word2vec, y_train, y_val, n_trials=30) 
print(metrics.classification_report(y_test, word2vec_predicted_NB))
print(metrics.confusion_matrix(y_test, word2vec_predicted_NB))

'''
print ("oversampling")
word2vec_over_sampling_predicted_NB= naive_bayes_gs(X_train_word2vec, X_val_word2vec, X_test_word2vec, y_train, y_val, n_trials=30) 
print(metrics.classification_report(y_test, word2vec_predicted_NB))
print(metrics.confusion_matrix(y_test, word2vec_predicted_NB))'''

word2vec_trained_predicted_NB= naive_bayes_gs(X_train_w2v_trained, X_val_w2v_trained, X_test_w2v_trained, y_train, y_val, n_trials=30) 
print(metrics.classification_report(y_test, word2vec_trained_predicted_NB))
print(metrics.confusion_matrix(y_test, word2vec_trained_predicted_NB))


[I 2025-11-12 12:41:11,396] A new study created in memory with name: no-name-deb6726e-f31c-41c7-ab6b-6e6cecbca165
[I 2025-11-12 12:41:11,528] Trial 0 finished with value: 0.4016961525191454 and parameters: {'var_smoothing': 5.483465937106425e-11}. Best is trial 0 with value: 0.4016961525191454.
[I 2025-11-12 12:41:11,557] Trial 4 finished with value: 0.4016961525191454 and parameters: {'var_smoothing': 0.0006054484518880846}. Best is trial 0 with value: 0.4016961525191454.
[I 2025-11-12 12:41:11,568] Trial 3 finished with value: 0.4016961525191454 and parameters: {'var_smoothing': 0.0003077393916607256}. Best is trial 0 with value: 0.4016961525191454.


----WORD2VEC ----


[I 2025-11-12 12:41:11,597] Trial 2 finished with value: 0.4016961525191454 and parameters: {'var_smoothing': 8.736382381409068e-08}. Best is trial 0 with value: 0.4016961525191454.
[I 2025-11-12 12:41:11,599] Trial 7 finished with value: 0.4016961525191454 and parameters: {'var_smoothing': 7.78171115115388e-07}. Best is trial 0 with value: 0.4016961525191454.
[I 2025-11-12 12:41:11,621] Trial 1 finished with value: 0.4016961525191454 and parameters: {'var_smoothing': 7.694122580730403e-05}. Best is trial 0 with value: 0.4016961525191454.
[I 2025-11-12 12:41:11,630] Trial 5 finished with value: 0.4016961525191454 and parameters: {'var_smoothing': 9.23114912610425e-09}. Best is trial 0 with value: 0.4016961525191454.
[I 2025-11-12 12:41:11,669] Trial 6 finished with value: 0.4016961525191454 and parameters: {'var_smoothing': 4.674728806800524e-10}. Best is trial 0 with value: 0.4016961525191454.
[I 2025-11-12 12:41:11,699] Trial 8 finished with value: 0.4016961525191454 and parameters: 

Melhores hiperparâmetros encontrados:
{'var_smoothing': 5.483465937106425e-11}
              precision    recall  f1-score   support

          -1       0.44      0.42      0.43       417
           0       0.84      0.32      0.47      1273
           1       0.11      0.62      0.18       168

    accuracy                           0.37      1858
   macro avg       0.46      0.46      0.36      1858
weighted avg       0.68      0.37      0.43      1858

[[175  44 198]
 [196 410 667]
 [ 30  33 105]]


[I 2025-11-12 12:41:12,539] Trial 3 finished with value: 0.5330195677046446 and parameters: {'var_smoothing': 1.0987447154035852e-11}. Best is trial 3 with value: 0.5330195677046446.
[I 2025-11-12 12:41:12,542] Trial 4 finished with value: 0.5330195677046446 and parameters: {'var_smoothing': 3.6475529411324714e-05}. Best is trial 3 with value: 0.5330195677046446.
[I 2025-11-12 12:41:12,557] Trial 0 finished with value: 0.5330195677046446 and parameters: {'var_smoothing': 2.967996208659967e-10}. Best is trial 3 with value: 0.5330195677046446.
[I 2025-11-12 12:41:12,571] Trial 2 finished with value: 0.5330195677046446 and parameters: {'var_smoothing': 8.662641407899121e-12}. Best is trial 3 with value: 0.5330195677046446.
[I 2025-11-12 12:41:12,611] Trial 7 finished with value: 0.5330195677046446 and parameters: {'var_smoothing': 2.4663464714344284e-10}. Best is trial 3 with value: 0.5330195677046446.
[I 2025-11-12 12:41:12,653] Trial 1 finished with value: 0.5330195677046446 and paramet

Melhores hiperparâmetros encontrados:
{'var_smoothing': 0.0002971419184786622}
              precision    recall  f1-score   support

          -1       0.67      0.65      0.66       417
           0       0.89      0.45      0.60      1273
           1       0.15      0.72      0.25       168

    accuracy                           0.52      1858
   macro avg       0.57      0.61      0.50      1858
weighted avg       0.78      0.52      0.58      1858

[[272  39 106]
 [115 579 579]
 [ 17  30 121]]


In [ ]:
#Glove
print ("----GLOVE ----")
glove_predicted_NB= naive_bayes_gs(X_train_glove, X_val_glove, X_test_glove, y_train, y_val, n_trials=30) 
lista_y = [str(c) for c in (test['classe'].unique())]
print(metrics.classification_report(
y_test, glove_predicted_NB, target_names=set(lista_y)))
print(metrics.confusion_matrix(y_test, glove_predicted_NB))

## SVM

In [35]:
def objective(trial, X_train, X_val, y_train, y_val):

    #Definir hiperparâmetros a testar 
    C = trial.suggest_float('C', 0.01, 10.0, log=True)
    kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly'])
    gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])
    degree = trial.suggest_int('degree', 2, 5) if kernel == 'poly' else 3

    #Criar modelo SVM 
    model = SVC(C=C, kernel=kernel, gamma=gamma, degree=degree)
    model = SVC(C=C, kernel=kernel, gamma=gamma, degree=degree)

    #treinar modelo com conjunto de treino
    model.fit(X_train, y_train)

    #prever no conjunto de validação
    y_pred = model.predict(X_val)

    #calcular f1
    f1 = f1_score(y_val, y_pred, average='macro')

    return f1

def svm(X_train, X_val, X_test, y_train, y_val, n_trials=30):
    # Chamar o Optuna para otimizar os parâmetros
    objeto_para_otimizar = optuna.create_study(direction='maximize')
    objeto_para_otimizar.optimize(lambda trial: objective(trial, X_train, X_val, y_train, y_val),
                   n_trials=n_trials, n_jobs=-1, show_progress_bar=False )

    print("Melhores hiperparâmetros encontrados:")
    print(objeto_para_otimizar.best_params)

    # Treinar o melhor modelo
    best_model = SVC(**objeto_para_otimizar.best_params)
    best_model.fit(X_train, y_train)

    #  Avaliar no conjunto de teste
    y_pred_test = best_model.predict(X_test)

    return y_pred_test

In [36]:
print ("----COUNT VECTORIZER ----")
count_vec_predicted_SVM= svm(X_train_count_vec, X_val_count_vec, X_test_count_vec, y_train, y_val, n_trials=30) 
lista_y = [str(c) for c in (test['classe'].unique())]
print(metrics.classification_report(
y_test, count_vec_predicted_SVM, target_names=set(lista_y)))
print(metrics.confusion_matrix(y_test, count_vec_predicted_SVM))

[I 2025-11-11 15:08:41,414] A new study created in memory with name: no-name-b383358a-4d3a-48f6-9f6e-1f4d8622cc66


----COUNT VECTORIZER ----


[I 2025-11-11 15:08:42,662] Trial 6 finished with value: 0.271047227926078 and parameters: {'C': 4.348664013511701, 'kernel': 'poly', 'gamma': 'auto', 'degree': 5}. Best is trial 6 with value: 0.271047227926078.
[I 2025-11-11 15:08:43,060] Trial 5 finished with value: 0.2728571162413656 and parameters: {'C': 0.4128041026288293, 'kernel': 'linear', 'gamma': 'scale'}. Best is trial 5 with value: 0.2728571162413656.
[I 2025-11-11 15:08:43,193] Trial 0 finished with value: 0.272029175469006 and parameters: {'C': 0.20074550984619463, 'kernel': 'linear', 'gamma': 'scale'}. Best is trial 5 with value: 0.2728571162413656.
[I 2025-11-11 15:08:43,251] Trial 7 finished with value: 0.26886900665290203 and parameters: {'C': 0.06227384659617249, 'kernel': 'linear', 'gamma': 'auto'}. Best is trial 5 with value: 0.2728571162413656.
[I 2025-11-11 15:08:43,403] Trial 2 finished with value: 0.2782224244910812 and parameters: {'C': 2.106516381508683, 'kernel': 'poly', 'gamma': 'scale', 'degree': 2}. Best 

Melhores hiperparâmetros encontrados:
{'C': 7.739094578920957, 'kernel': 'linear', 'gamma': 'auto'}
              precision    recall  f1-score   support

           1       0.20      0.12      0.15       417
           0       0.68      0.78      0.73      1273
          -1       0.13      0.11      0.12       168

    accuracy                           0.57      1858
   macro avg       0.34      0.34      0.33      1858
weighted avg       0.52      0.57      0.54      1858

[[ 50 341  26]
 [173 996 104]
 [ 26 123  19]]


In [37]:
print ("---- TF-IDF ----")
tfidf_vec_predicted_SVM= svm(X_train_tfidf_vec, X_val_tfidf_vec, X_test_tfidf_vec, y_train, y_val, n_trials=30) 
lista_y = [str(c) for c in (train['classe'].unique())]
print(metrics.classification_report(
y_test, tfidf_vec_predicted_SVM, target_names=set(lista_y)))
print(metrics.confusion_matrix(y_test, tfidf_vec_predicted_SVM))

[I 2025-11-11 15:08:54,499] A new study created in memory with name: no-name-d4b27d55-029c-47c2-b536-e77953e56d19


---- TF-IDF ----


[I 2025-11-11 15:08:55,173] Trial 1 finished with value: 0.271047227926078 and parameters: {'C': 0.012325279805459869, 'kernel': 'poly', 'gamma': 'auto', 'degree': 3}. Best is trial 1 with value: 0.271047227926078.
[I 2025-11-11 15:08:55,293] Trial 0 finished with value: 0.271047227926078 and parameters: {'C': 0.9277300707410308, 'kernel': 'poly', 'gamma': 'auto', 'degree': 4}. Best is trial 1 with value: 0.271047227926078.
[I 2025-11-11 15:08:55,432] Trial 4 finished with value: 0.271047227926078 and parameters: {'C': 0.012977477690234215, 'kernel': 'rbf', 'gamma': 'auto'}. Best is trial 1 with value: 0.271047227926078.
[I 2025-11-11 15:08:55,467] Trial 5 finished with value: 0.271047227926078 and parameters: {'C': 0.21656092956323936, 'kernel': 'rbf', 'gamma': 'auto'}. Best is trial 1 with value: 0.271047227926078.
[I 2025-11-11 15:08:55,671] Trial 7 finished with value: 0.280653324131585 and parameters: {'C': 0.27235873079065787, 'kernel': 'linear', 'gamma': 'auto'}. Best is trial 7

Melhores hiperparâmetros encontrados:
{'C': 2.653216629374672, 'kernel': 'linear', 'gamma': 'scale'}
              precision    recall  f1-score   support

           1       0.23      0.11      0.14       417
           0       0.69      0.88      0.77      1273
          -1       0.11      0.02      0.03       168

    accuracy                           0.63      1858
   macro avg       0.34      0.33      0.32      1858
weighted avg       0.53      0.63      0.56      1858

[[  44  365    8]
 [ 135 1121   17]
 [  15  150    3]]


In [39]:
print ("----WORD2VEC ----")
word2vec_predicted_SVM= svm(X_train_word2vec, X_val_word2vec, X_test_word2vec, y_train, y_val, n_trials=30) 
lista_y = [str(c) for c in (test['classe'].unique())]
print(metrics.classification_report(
y_test, word2vec_predicted_SVM, target_names=set(lista_y)))
print(metrics.confusion_matrix(y_test, word2vec_predicted_SVM))

[I 2025-11-11 15:09:35,410] A new study created in memory with name: no-name-c5f44939-d87b-45d7-b70b-c822e11faef4


----WORD2VEC ----


[I 2025-11-11 15:09:37,628] Trial 1 finished with value: 0.271047227926078 and parameters: {'C': 0.3853407159473238, 'kernel': 'poly', 'gamma': 'auto', 'degree': 3}. Best is trial 1 with value: 0.271047227926078.
[I 2025-11-11 15:09:37,951] Trial 4 finished with value: 0.271047227926078 and parameters: {'C': 0.038993382676786, 'kernel': 'poly', 'gamma': 'auto', 'degree': 3}. Best is trial 1 with value: 0.271047227926078.
[I 2025-11-11 15:09:38,323] Trial 6 finished with value: 0.271047227926078 and parameters: {'C': 0.013677459196437055, 'kernel': 'linear', 'gamma': 'auto'}. Best is trial 1 with value: 0.271047227926078.
[I 2025-11-11 15:09:38,518] Trial 5 finished with value: 0.4338411420906391 and parameters: {'C': 0.8597224524711686, 'kernel': 'linear', 'gamma': 'auto'}. Best is trial 5 with value: 0.4338411420906391.
[I 2025-11-11 15:09:38,936] Trial 2 finished with value: 0.271047227926078 and parameters: {'C': 1.365233127757608, 'kernel': 'rbf', 'gamma': 'auto'}. Best is trial 5 

Melhores hiperparâmetros encontrados:
{'C': 9.082446685549941, 'kernel': 'rbf', 'gamma': 'scale'}
              precision    recall  f1-score   support

           1       0.59      0.44      0.51       417
           0       0.76      0.90      0.83      1273
          -1       0.33      0.09      0.14       168

    accuracy                           0.72      1858
   macro avg       0.56      0.48      0.49      1858
weighted avg       0.69      0.72      0.69      1858

[[ 185  225    7]
 [ 104 1146   23]
 [  22  131   15]]


## KNN

In [23]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

def KNN(X_train, y_train, X_val, y_val, X_test):
    k_values = range(1, 200,10)  # Determinar  intervalo de valores a testar para k
    best_k = 1
    best_accuracy = 0
    
    k_list = []
    accuracy_list = [] 

    for k in k_values:

        knn = KNeighborsClassifier(n_neighbors=k)  
        knn.fit(X_train, y_train)

        y_pred_KNN = knn.predict(X_val) 
        accuracy = accuracy_score(y_val, y_pred_KNN)  

        k_list.append(k)
        accuracy_list.append(accuracy)  

        if accuracy > best_accuracy: 
            best_accuracy = accuracy
            best_k = k

    knn = KNeighborsClassifier(n_neighbors=best_k)  
    knn.fit(X_train, y_train)

    y_pred_KNN= knn.predict(X_test) 
    print(f"Best k: {best_k}")

    return y_pred_KNN


In [ ]:
# Count Vectorizer
print ("----COUNT VECTORIZER ----")
count_vec_predicted_KNN= KNN(X_train_count_vec, y_train, X_val_count_vec, y_val, X_test_count_vec) 
lista_y = [str(c) for c in (test['classe'].unique())]
print(metrics.classification_report(
y_test, count_vec_predicted_KNN, target_names=set(lista_y)))
print(metrics.confusion_matrix(y_test, count_vec_predicted_KNN))

----COUNT VECTORIZER ----
Best k: 151
              precision    recall  f1-score   support

          -1       0.00      0.00      0.00       417
           1       0.68      0.98      0.80      1273
           0       0.00      0.00      0.00       168

    accuracy                           0.67      1858
   macro avg       0.23      0.33      0.27      1858
weighted avg       0.47      0.67      0.55      1858

[[   0  417    0]
 [  29 1244    0]
 [   0  168    0]]


c:\Users\didia\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\didia\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\didia\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


In [ ]:
#TF-IDF
print ("----TF-IDF ----")
tfidf_vec_predicted_KNN= KNN(X_train_tfidf_vec, y_train, X_val_tfidf_vec, y_val, X_test_tfidf_vec) 
lista_y = [str(c) for c in (test['classe'].unique())]
print(metrics.classification_report(
y_test, tfidf_vec_predicted_KNN, target_names=set(lista_y)))
print(metrics.confusion_matrix(y_test, tfidf_vec_predicted_KNN))

In [ ]:
#Word2vec
print ("----WORD2VEC ----")
word2vec_predicted_KNN= KNN(X_train_word2vec, y_train, X_val_word2vec, y_val, X_test_word2vec) 
lista_y = [str(c) for c in (test['classe'].unique())]
print(metrics.classification_report(
y_test, word2vec_predicted_KNN, target_names=set(lista_y)))
print(metrics.confusion_matrix(y_test, word2vec_predicted_KNN))

----COUNT VECTORIZER ----
Best k: 151
              precision    recall  f1-score   support

          -1       0.00      0.00      0.00       417
           1       0.68      0.98      0.80      1273
           0       0.00      0.00      0.00       168

    accuracy                           0.67      1858
   macro avg       0.23      0.33      0.27      1858
weighted avg       0.47      0.67      0.55      1858

[[   0  417    0]
 [  29 1244    0]
 [   0  168    0]]
----WORD2VEC ----


c:\Users\didia\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\didia\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\didia\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


Best k: 11
              precision    recall  f1-score   support

          -1       0.62      0.27      0.37       417
           1       0.73      0.95      0.82      1273
           0       0.20      0.01      0.02       168

    accuracy                           0.71      1858
   macro avg       0.51      0.41      0.41      1858
weighted avg       0.65      0.71      0.65      1858

[[ 112  304    1]
 [  56 1210    7]
 [  13  153    2]]


In [ ]:
#Glove

## Decison Tree

In [25]:
#Criar um classificador de árvore de decisão baseado em entropia
from sklearn.tree import DecisionTreeClassifier

def decision_tree(X_train, y_train, X_val, y_val, X_test):
    alphas = np.linspace(0.000, 0.01, 20)  # Testar 20 valores entre 0 e 0.01
    best_alpha = 0
    best_acc = 0

    for alpha in alphas:
        clf = DecisionTreeClassifier(criterion='entropy', ccp_alpha=alpha, random_state=42)
        clf.fit(X_train, y_train)
        y_pred_val = clf.predict(X_val)
        acc = accuracy_score(y_val, y_pred_val)

        if acc > best_acc:
            best_acc = acc
            best_alpha = alpha

    print(f"Melhor alpha: {best_alpha:.4f} ")

    # Treinar modelo final com o melhor alpha
    final_clf = DecisionTreeClassifier(criterion='entropy', ccp_alpha=best_alpha, random_state=42)
    final_clf.fit(X_train, y_train)
    y_pred_test = final_clf.predict(X_test)

    return y_pred_test

In [ ]:
# Count Vectorizer
print ("----COUNT VECTORIZER ----")
count_vec_predicted_DT= decision_tree(X_train_count_vec, y_train, X_val_count_vec, y_val, X_test_count_vec) 
lista_y = [str(c) for c in (test['classe'].unique())]
print(metrics.classification_report(
y_test, count_vec_predicted_DT, target_names=set(lista_y)))
print(metrics.confusion_matrix(y_test, count_vec_predicted_DT))


In [ ]:
#TF-IDF
print ("----TF-IDF ----")
tfidf_vec_predicted_DT= decision_tree(X_train_tfidf_vec, y_train, X_val_tfidf_vec, y_val, X_test_tfidf_vec) 
lista_y = [str(c) for c in (test['classe'].unique())]
print(metrics.classification_report(
y_test, tfidf_vec_predicted_DT, target_names=set(lista_y)))
print(metrics.confusion_matrix(y_test, tfidf_vec_predicted_DT))

In [ ]:
#Word2vec
print ("----WORD2VEC ----")
word2vec_predicted_DT= decision_tree(X_train_word2vec, y_train, X_val_word2vec, y_val, X_test_word2vec) 
lista_y = [str(c) for c in (test['classe'].unique())]
print(metrics.classification_report(
y_test, word2vec_predicted_DT, target_names=set(lista_y)))
print(metrics.confusion_matrix(y_test, word2vec_predicted_DT))

----COUNT VECTORIZER ----
Melhor alpha: 0.0037 


c:\Users\didia\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\didia\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\didia\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


              precision    recall  f1-score   support

          -1       0.08      0.03      0.04       417
           1       0.67      0.90      0.77      1273
           0       0.00      0.00      0.00       168

    accuracy                           0.62      1858
   macro avg       0.25      0.31      0.27      1858
weighted avg       0.48      0.62      0.54      1858

[[  12  405    0]
 [ 126 1147    0]
 [   4  164    0]]
----COUNT VECTORIZER ----
Melhor alpha: 0.0047 
              precision    recall  f1-score   support

          -1       0.41      0.30      0.35       417
           1       0.72      0.88      0.79      1273
           0       0.00      0.00      0.00       168

    accuracy                           0.67      1858
   macro avg       0.38      0.39      0.38      1858
weighted avg       0.59      0.67      0.62      1858

[[ 126  291    0]
 [ 153 1120    0]
 [  26  142    0]]


c:\Users\didia\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\didia\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\didia\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


In [ ]:
#Glove

## Random Florest

In [ ]:
import optuna
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, confusion_matrix, classification_report, make_scorer

def objective(X_train, y_train, X_val, y_val, trial):
    n_estimators = trial.suggest_categorical('n_estimators', [50, 100, 200, 300])
    max_depth = trial.suggest_categorical('max_depth', [5, 10, 15, 20])
    max_features = trial.suggest_categorical('max_features', ['sqrt', 'log2', None])
    criterion = trial.suggest_categorical('criterion', ['gini', 'entropy'])

    clf = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        max_features=max_features,
        criterion=criterion,
        random_state=42
    )

    clf.fit(X_train, y_train)

    predictions_rf = clf.predict(X_val)

    f1 = f1_score(y_val, predictions_rf, pos_label=1,average='macro')
    
    return f1

def random_florest(X_train, y_train, X_val, y_val, X_test, y_test):
    study = optuna.create_study(direction='maximize')
    study.optimize(lambda trial: objective(X_train, y_train, X_val, y_val, trial),
                   n_trials=50, n_jobs=-1)

    print("Melhores hiperparâmetros Random Forest:")
    print(study.best_params)

    best_rf = RandomForestClassifier(**study.best_params, random_state=42)
    best_rf.fit(X_train, y_train)

    predictions_rf = best_rf.predict(X_test)
    
    return predictions_rf


In [ ]:
# Count Vectorizer
print ("----COUNT VECTORIZER ----")
count_vec_predicted_DT= random_florest(X_train_count_vec, y_train, X_val_count_vec, y_val, X_test_count_vec) 
lista_y = [str(c) for c in (test['classe'].unique())]
print(metrics.classification_report(
y_test, count_vec_predicted_DT, target_names=set(lista_y)))
print(metrics.confusion_matrix(y_test, count_vec_predicted_DT))

In [ ]:
#TF-IDF
print ("----TF-IDF ----")
tfidf_vec_predicted_DT= random_florest(X_train_tfidf_vec, y_train, X_val_tfidf_vec, y_val, X_test_tfidf_vec) 
lista_y = [str(c) for c in (test['classe'].unique())]
print(metrics.classification_report(
y_test, tfidf_vec_predicted_DT, target_names=set(lista_y)))
print(metrics.confusion_matrix(y_test, tfidf_vec_predicted_DT))

In [ ]:
#Word2vec
print ("----WORD2VEC ----")
word2vec_predicted_DT= random_florest(X_train_word2vec, y_train, X_val_word2vec, y_val, X_test_word2vec) 
lista_y = [str(c) for c in (test['classe'].unique())]
print(metrics.classification_report(
y_test, word2vec_predicted_DT, target_names=set(lista_y)))
print(metrics.confusion_matrix(y_test, word2vec_predicted_DT))

In [ ]:
#Glove

## Neural network

In [6]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score
import optuna

def objective(X_train, y_train, X_val, y_val, trial):
    hidden_layer_sizes = trial.suggest_categorical('hidden_layer_sizes', [(50,), (100,), (100, 50)])
    activation = trial.suggest_categorical('activation', ['relu', 'tanh'])
    learning_rate_init = trial.suggest_float('learning_rate_init', 1e-4, 1e-2, log=True)

    clf = MLPClassifier(
        hidden_layer_sizes=hidden_layer_sizes,
        activation=activation,
        learning_rate_init=learning_rate_init,
        max_iter=300,
        random_state=42
    )

    clf.fit(X_train, y_train)
    preds = clf.predict(X_val)
    f1 = f1_score(y_val, preds, average='macro')
    return f1

def neural_network (X_train, y_train, X_val, y_val, X_test, y_test):
    study = optuna.create_study(direction='maximize')
    study.optimize(lambda trial: objective(X_train, y_train, X_val, y_val, trial), n_trials=30)
    print("Melhores parâmetros:", study.best_params)

    best_params = study.best_params
    best_model = MLPClassifier(
        hidden_layer_sizes=best_params['hidden_layer_sizes'],
        activation=best_params['activation'],
        learning_rate_init=best_params['learning_rate_init'],
        max_iter=300,
        random_state=42
    )

    best_model.fit(X_train, y_train)
    previsoes = best_model.predict(X_test)
    
    return previsoes


In [ ]:
# Count Vectorizer
print ("----COUNT VECTORIZER ----")
count_vec_predicted_KNN= neural_network(X_train_count_vec, y_train, X_val_count_vec, y_val, X_test_count_vec) 
lista_y = [str(c) for c in (test['classe'].unique())]
print(metrics.classification_report(
y_test, count_vec_predicted_KNN, target_names=set(lista_y)))
print(metrics.confusion_matrix(y_test, count_vec_predicted_KNN))

In [ ]:
#TF-IDF
print ("----TF-IDF ----")
tfidf_vec_predicted_DT= neural_network(X_train_tfidf_vec, y_train, X_val_tfidf_vec, y_val, X_test_tfidf_vec) 
lista_y = [str(c) for c in (test['classe'].unique())]
print(metrics.classification_report(
y_test, tfidf_vec_predicted_DT, target_names=set(lista_y)))
print(metrics.confusion_matrix(y_test, tfidf_vec_predicted_DT))

In [ ]:
#Word2vec
print ("----WORD2VEC ----")
word2vec_predicted_DT= neural_network(X_train_word2vec, y_train, X_val_word2vec, y_val, X_test_word2vec) 
lista_y = [str(c) for c in (test['classe'].unique())]
print(metrics.classification_report(
y_test, word2vec_predicted_DT, target_names=set(lista_y)))
print(metrics.confusion_matrix(y_test, word2vec_predicted_DT))

In [ ]:
#Glove

## Regressão Logística

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
import optuna

def objective(X_train, y_train, X_val, y_val, trial):
    # Hiperparâmetros a otimizar
    C = trial.suggest_float('C', 1e-3, 1e3, log=True)
    penalty = trial.suggest_categorical('penalty', ['l1', 'l2'])
    solver = trial.suggest_categorical('solver', ['liblinear', 'saga'])  # compatíveis com L1 e L2
    max_iter = trial.suggest_int('max_iter', 100, 1000)

    model = LogisticRegression(
        C=C,
        penalty=penalty,
        solver=solver,
        max_iter=max_iter,
        random_state=42
    )

    model.fit(X_train, y_train)
    preds = model.predict(X_val)

    f1 = f1_score(y_val, preds, average='macro')
    return f1


def logistic_regression(X_train, y_train, X_val, y_val, X_test, y_test):
    study = optuna.create_study(direction='maximize')
    study.optimize(lambda trial: objective(X_train, y_train, X_val, y_val, trial), n_trials=30)

    print("Melhores parâmetros encontrados:")
    print(study.best_params)

    best_params = study.best_params
    best_model = LogisticRegression(
        **best_params,
        random_state=42
    )

    best_model.fit(X_train, y_train)

    previsoes = best_model.predict(X_test)

    return previsoes


In [ ]:
# Count Vectorizer
print ("----COUNT VECTORIZER ----")
count_vec_predicted_KNN= logistic_regression(X_train_count_vec, y_train, X_val_count_vec, y_val, X_test_count_vec) 
lista_y = [str(c) for c in (test['classe'].unique())]
print(metrics.classification_report(
y_test, count_vec_predicted_KNN, target_names=set(lista_y)))
print(metrics.confusion_matrix(y_test, count_vec_predicted_KNN))

In [ ]:
#TF-IDF
print ("----TF-IDF ----")
tfidf_vec_predicted_DT= logistic_regression(X_train_tfidf_vec, y_train, X_val_tfidf_vec, y_val, X_test_tfidf_vec) 
lista_y = [str(c) for c in (test['classe'].unique())]
print(metrics.classification_report(
y_test, tfidf_vec_predicted_DT, target_names=set(lista_y)))
print(metrics.confusion_matrix(y_test, tfidf_vec_predicted_DT))

In [ ]:
#Word2vec
print ("----WORD2VEC ----")
word2vec_predicted_DT= neural_network(X_train_word2vec, y_train, X_val_word2vec, y_val, X_test_word2vec) 
lista_y = [str(c) for c in (test['classe'].unique())]
print(metrics.classification_report(
y_test, word2vec_predicted_DT, target_names=set(lista_y)))
print(metrics.confusion_matrix(y_test, word2vec_predicted_DT))

In [ ]:
#Glove